#Custom ResNets with Keras

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## A traditional CNN (as a benchmark on Fashion CIFAR10)

In [11]:
# import CIFAR10
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(45000, 32, 32, 3) (5000, 32, 32, 3) (10000, 32, 32, 3)
(45000, 1) (5000, 1) (10000, 1)


In [107]:
# assemble a simple CNN
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same", input_shape=[32, 32, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
])

In [108]:
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_183 (Conv2D)          (None, 32, 32, 64)        9472      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_184 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_185 (Conv2D)          (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_186 (Conv2D)          (None, 8, 8, 256)         295168    
_________________________________________________________________
conv2d_187 (Conv2D)          (None, 8, 8, 256)       

In [110]:
# compile and train the model
optimizer = tf.keras.optimizers.Adagrad()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
1407/1407 [==============================] - 25s 18ms/step - loss: 1.9614 - accuracy: 0.2840 - val_loss: 1.7877 - val_accuracy: 0.3732
Epoch 2/20
1407/1407 [==============================] - 25s 18ms/step - loss: 1.8951 - accuracy: 0.3092 - val_loss: 1.7170 - val_accuracy: 0.3998
Epoch 3/20
1407/1407 [==============================] - 25s 18ms/step - loss: 1.8507 - accuracy: 0.3252 - val_loss: 1.7010 - val_accuracy: 0.4072
Epoch 4/20
1407/1407 [==============================] - 25s 17ms/step - loss: 1.8176 - accuracy: 0.3394 - val_loss: 1.6562 - val_accuracy: 0.4234
Epoch 5/20
1407/1407 [==============================] - 25s 17ms/step - loss: 1.7926 - accuracy: 0.3472 - val_loss: 1.6042 - val_accuracy: 0.4408
Epoch 6/20
1407/1407 [==============================] - 25s 18ms/step - loss: 1.7648 - accuracy: 0.3587 - val_loss: 1.5887 - val_accuracy: 0.4466
Epoch 7/20
1407/1407 [==============================] - 25s 17ms/step - loss: 1.7450 - accuracy: 0.3679 - val_loss: 1.5598 -

In [7]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.2114 - accuracy: 0.5726


[1.2113527059555054, 0.5726000070571899]

## A shallow network with Inception blocks

In [12]:
# a custom residual unit, obtained by subclassing keras.layers.Layer
class Inception(keras.layers.Layer):
    def __init__(self, filters, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.activation = keras.activations.get(activation)
        self.layers_0 = [
        keras.layers.Conv2D(self.filters[0], 1, strides=1, padding="same", use_bias=False),
        self.activation]
        self.layers_1 = [
        keras.layers.Conv2D(self.filters[1], 1, strides=1, padding="same", use_bias=False),
        self.activation, 
        keras.layers.Conv2D(self.filters[2], 3, strides=1, padding="same", use_bias=False),
        self.activation]
        self.layers_2 = [
        keras.layers.Conv2D(self.filters[3], 1, strides=1, padding="same", use_bias=False),
        self.activation, 
        keras.layers.Conv2D(self.filters[4], 5, strides=1, padding="same", use_bias=False),
        self.activation]
        self.layers_3 = [
        keras.layers.MaxPooling2D(pool_size=2, strides=1, padding="same"), 
        keras.layers.Conv2D(self.filters[5], 5, strides=1, padding="same", use_bias=False),
        self.activation]
        
    def call(self, inputs):
        Z0 = inputs
        Z1 = inputs
        Z2 = inputs
        Z3 = inputs

        for layer in self.layers_0:
            Z0 = layer(Z0)
        for layer in self.layers_1:
            Z1 = layer(Z1)
        for layer in self.layers_2:
            Z2 = layer(Z2)
        for layer in self.layers_3:
            Z3 = layer(Z3)
        
        return tf.concat([Z0, Z1, Z2, Z3], axis=3)

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_72 (Conv2D)           (None, 16, 16, 64)        9408      
_________________________________________________________________
batch_normalization_72 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
res_block_32 (ResBlock)      (None, 8, 8, 64)          0         
_________________________________________________________________
res_block_33 (ResBlock)      (None, 8, 8, 64)          0         
_________________________________________________________________
res_block_34 (ResBlock)      (None, 8, 8, 64)         

In [14]:
# assemble a simple CNN
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 5, activation="relu", padding="same", input_shape=[32, 32, 3]),
    keras.layers.MaxPooling2D(pool_size=3, strides=1, padding="same"),
    Inception([32, 48, 64, 8, 16, 16]),
    keras.layers.Conv2D(32, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(pool_size=3, strides=1, padding="same"),
    Inception([32, 48, 64, 8, 16, 16]),
    keras.layers.GlobalAveragePooling2D(),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
])

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_108 (Conv2D)          (None, 32, 32, 32)        2432      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
inception (Inception)        (None, 32, 32, 128)       46464     
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 32, 32, 32)        36896     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
inception_1 (Inception)      (None, 32, 32, 128)       46464     
_________________________________________________________________
global_average_pooling2d_3 ( (None, 128)              

In [16]:
# compile and train the model
optimizer = tf.keras.optimizers.Adagrad()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
  61/1407 [>.............................] - ETA: 47s - loss: 2.3011 - accuracy: 0.1122

KeyboardInterrupt: ignored

## Mini ResNet

In [2]:
# residual block
class ResBlock(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, 
                                padding="same", use_bias=False),
            keras.layers.BatchNormalization(), 
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, 
                                padding="same", use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides,
                                    padding="same", use_bias=False),
                keras.layers.BatchNormalization()]
        
        def call(self, inputs):
            Z = inputs
            for layer in self.main_layers:
                Z = layer(Z)
            skip_Z = inputs
            for layer in self.skip_layers:
                skip_Z = layer(skip_Z)
            return self.activation(Z + skip_Z)

In [3]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[32, 32, 3], 
                              padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResBlock(filters, strides=strides)) 
    prev_filters = filters   
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 64)        9408      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 64)          0         
_________________________________________________________________
res_block (ResBlock)         (None, 8, 8, 64)          0         
_________________________________________________________________
res_block_1 (ResBlock)       (None, 8, 8, 64)          0         
_________________________________________________________________
res_block_2 (ResBlock)       (None, 8, 8, 64)          0

In [20]:
# compile and train the model
optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid))

Epoch 1/50
1407/1407 [==============================] - 9s 7ms/step - loss: 1.8126 - accuracy: 0.3665 - val_loss: 2.5798 - val_accuracy: 0.1602
Epoch 2/50
  60/1407 [>.............................] - ETA: 6s - loss: 1.6776 - accuracy: 0.4161

KeyboardInterrupt: ignored